In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import category_encoders as ce

In [3]:
df = pd.read_csv('/Users/ethanchu/Downloads/osfstorage-archive/stats.anx.merged.cleaned.june4.v10.csv')

In [18]:
display(df.head(10))

,id,program.type,age,ethnicity.r,major,dal.course.code,se.1,se.2,se.3,se.4,...,frost.ps,frost.com,frost.da,stat.anx.tc,stat.anx.i,stat.anx.ah,stat.anx.ws,stat.anx.fst,stat.anx.sc,math.anx
0,12490341199,0,21,White,Psychology,"PSYO 2501, STAT 1060",4.0,3.0,4.0,4.0,...,4.00,4.4,3.25,4.666667,3.666667,5.0,3.666667,3.666667,2.0,4.111111
1,12489089323,0,22,Middle Eastern,Biology,STAT 1060,5.0,5.0,5.0,4.0,...,4.00,2.4,1.25,3.666667,1.000000,1.0,1.000000,1.000000,1.0,1.333333
2,12488861061,0,16,Black,Acting,NaN,4.0,5.0,5.0,5.0,...,3.25,2.4,4.00,4.333333,2.333333,1.0,2.000000,2.000000,2.0,1.888889
3,12487149081,0,19,White,Pharmacy,NaN,5.0,4.0,4.0,5.0,...,3.75,2.6,3.25,2.000000,1.333333,2.5,2.666667,2.666667,1.5,1.555556
4,12486771921,0,18,White,Undecided,STAT 1060,4.0,3.0,4.0,4.0,...,3.75,2.8,3.25,4.666667,3.333333,4.0,2.000000,2.333333,1.0,3.222222
5,12486512165,0,19,White,neuroscience,stat 1060,4.0,4.0,3.0,5.0,...,5.00,3.4,3.50,4.333333,4.000000,3.5,4.333333,2.666667,2.5,4.111111
6,12484222078,0,21,Hispanic/Latinx,Microbiology & Immunology,STAT1060,4.0,4.0,4.0,4.0,...,4.25,3.6,3.75,4.333333,3.333333,5.0,1.000000,1.000000,1.0,3.111111
7,12484106653,0,19,White,undecided,NaN,4.0,4.0,5.0,4.0,...,2.00,2.4,1.75,NaN,NaN,3.0,2.333333,2.000000,2.5,2.111111
8,12483856411,0,18,White,undecided,STAT 1060,4.0,3.0,4.0,4.0,...,2.25,2.6,2.75,3.666667,3.333333,3.0,3.000000,3.333333,3.5,3.555556
9,12481184335,0,28,Mixed/Other,undecided,NaN,5.0,5.0,5.0,5.0,...,2.25,1.0,1.75,4.666667,1.666667,1.0,1.000000,1.000000,1.0,2.000000


In [19]:
df_missing = df.isnull().sum()
df_missing = df_missing[df_missing > 0]
print(df_missing)

dal.course.code    198
se.1                 2
se.2                 1
se.3                 6
se.4                 6
                  ... 
stat.anx.ah          1
stat.anx.ws          1
stat.anx.fst         3
stat.anx.sc          1
math.anx             2
Length: 75, dtype: int64


In [20]:
print(df.duplicated().sum())

0


In [ ]:
# Drop irrelevant columns

In [6]:
df_new = df.drop(['id', 'dal.course.code'], axis=1)

In [7]:
display(df_new.head(10))

,program.type,age,ethnicity.r,major,se.1,se.2,se.3,se.4,se.5,se.6,...,frost.ps,frost.com,frost.da,stat.anx.tc,stat.anx.i,stat.anx.ah,stat.anx.ws,stat.anx.fst,stat.anx.sc,math.anx
0,0,21,White,Psychology,4.0,3.0,4.0,4.0,4.0,4.0,...,4.00,4.4,3.25,4.666667,3.666667,5.0,3.666667,3.666667,2.0,4.111111
1,0,22,Middle Eastern,Biology,5.0,5.0,5.0,4.0,4.0,5.0,...,4.00,2.4,1.25,3.666667,1.000000,1.0,1.000000,1.000000,1.0,1.333333
2,0,16,Black,Acting,4.0,5.0,5.0,5.0,5.0,5.0,...,3.25,2.4,4.00,4.333333,2.333333,1.0,2.000000,2.000000,2.0,1.888889
3,0,19,White,Pharmacy,5.0,4.0,4.0,5.0,4.0,4.0,...,3.75,2.6,3.25,2.000000,1.333333,2.5,2.666667,2.666667,1.5,1.555556
4,0,18,White,Undecided,4.0,3.0,4.0,4.0,4.0,4.0,...,3.75,2.8,3.25,4.666667,3.333333,4.0,2.000000,2.333333,1.0,3.222222
5,0,19,White,neuroscience,4.0,4.0,3.0,5.0,5.0,5.0,...,5.00,3.4,3.50,4.333333,4.000000,3.5,4.333333,2.666667,2.5,4.111111
6,0,21,Hispanic/Latinx,Microbiology & Immunology,4.0,4.0,4.0,4.0,5.0,5.0,...,4.25,3.6,3.75,4.333333,3.333333,5.0,1.000000,1.000000,1.0,3.111111
7,0,19,White,undecided,4.0,4.0,5.0,4.0,5.0,4.0,...,2.00,2.4,1.75,NaN,NaN,3.0,2.333333,2.000000,2.5,2.111111
8,0,18,White,undecided,4.0,3.0,4.0,4.0,4.0,4.0,...,2.25,2.6,2.75,3.666667,3.333333,3.0,3.000000,3.333333,3.5,3.555556
9,0,28,Mixed/Other,undecided,5.0,5.0,5.0,5.0,5.0,5.0,...,2.25,1.0,1.75,4.666667,1.666667,1.0,1.000000,1.000000,1.0,2.000000


In [8]:
df_new.to_csv('anxiety.csv', index=False)

In [137]:
df_new = df_new.ffill()

In [138]:
df_new.isnull().sum()

program.type    0
age             0
ethnicity.r     0
major           0
se.1            0
               ..
stat.anx.ah     0
stat.anx.ws     0
stat.anx.fst    0
stat.anx.sc     0
math.anx        0
Length: 95, dtype: int64

In [ ]:
# Target encode categorical columns

In [139]:
categorical_col = df_new.select_dtypes(include=['object']).columns.to_list()

In [140]:
encoder = ce.TargetEncoder(cols=categorical_col)

X_encoded = encoder.fit_transform(df_new.drop(columns=['math.anx', 'math.anx.1', 'math.anx.2', 'math.anx.3', 'math.anx.4', 'math.anx.5', 'math.anx.6', 'math.anx.7', 'math.anx.8', 'math.anx.9']), df_new['math.anx'])

In [141]:
display(X_encoded)

,program.type,age,ethnicity.r,major,se.1,se.2,se.3,se.4,se.5,se.6,...,perf.sp,frost.ps,frost.com,frost.da,stat.anx.tc,stat.anx.i,stat.anx.ah,stat.anx.ws,stat.anx.fst,stat.anx.sc
0,0,21,2.603117,2.867690,4.0,3.0,4.0,4.0,4.0,4.0,...,7.000000,4.00,4.4,3.25,4.666667,3.666667,5.0,3.666667,3.666667,2.0
1,0,22,2.703037,2.505334,5.0,5.0,5.0,4.0,4.0,5.0,...,7.000000,4.00,2.4,1.25,3.666667,1.000000,1.0,1.000000,1.000000,1.0
2,0,16,2.632999,2.519285,4.0,5.0,5.0,5.0,5.0,5.0,...,6.666667,3.25,2.4,4.00,4.333333,2.333333,1.0,2.000000,2.000000,2.0
3,0,19,2.603117,2.475916,5.0,4.0,4.0,5.0,4.0,4.0,...,4.333333,3.75,2.6,3.25,2.000000,1.333333,2.5,2.666667,2.666667,1.5
4,0,18,2.603117,2.707339,4.0,3.0,4.0,4.0,4.0,4.0,...,7.000000,3.75,2.8,3.25,4.666667,3.333333,4.0,2.000000,2.333333,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,1,22,2.603117,2.577111,4.0,2.0,4.0,2.0,4.0,3.0,...,6.333333,4.25,2.8,3.25,2.666667,1.666667,3.5,1.333333,2.333333,1.0
449,1,26,2.603117,2.432546,4.0,4.0,4.0,3.0,4.0,3.0,...,3.333333,2.75,2.2,3.00,3.666667,2.000000,2.0,1.333333,3.000000,3.0
450,1,28,2.727614,2.461459,4.0,3.0,4.0,5.0,4.0,5.0,...,3.666667,2.75,1.0,3.75,3.333333,2.666667,2.5,1.000000,1.000000,1.0
451,1,19,2.603117,2.608484,4.0,3.0,5.0,2.0,5.0,4.0,...,5.666667,4.25,3.0,3.25,4.333333,3.666667,2.0,1.333333,2.000000,1.0


In [9]:
numeric = df_new.select_dtypes(include=['int64', 'float64'])
print(numeric)

     program.type  age  se.1  se.2  se.3  se.4  se.5  se.6  se.7  se.8  ...  \
0               0   21   4.0   3.0   4.0   4.0   4.0   4.0   3.0   3.0  ...   
1               0   22   5.0   5.0   5.0   4.0   4.0   5.0   5.0   3.0  ...   
2               0   16   4.0   5.0   5.0   5.0   5.0   5.0   4.0   5.0  ...   
3               0   19   5.0   4.0   4.0   5.0   4.0   4.0   3.0   3.0  ...   
4               0   18   4.0   3.0   4.0   4.0   4.0   4.0   4.0   4.0  ...   
..            ...  ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
448             1   22   4.0   2.0   4.0   2.0   4.0   3.0   3.0   4.0  ...   
449             1   26   4.0   4.0   4.0   3.0   4.0   3.0   3.0   4.0  ...   
450             1   28   4.0   3.0   4.0   5.0   4.0   5.0   5.0   4.0  ...   
451             1   19   4.0   3.0   5.0   2.0   5.0   4.0   4.0   4.0  ...   
452             1   21   2.0   2.0   4.0   3.0   2.0   4.0   2.0   3.0  ...   

     frost.ps  frost.com  frost.da  stat.anx.tc  st

In [10]:
numeric = numeric.ffill()

In [11]:
print('Number of nulls:', numeric.isnull().sum().sum())

Number of nulls: 0


In [12]:
df_new.columns

Index(['program.type', 'age', 'ethnicity.r', 'major', 'se.1', 'se.2', 'se.3',
       'se.4', 'se.5', 'se.6', 'se.7', 'se.8', 'asi.1', 'asi.2', 'asi.3',
       'asi.4', 'asi.5', 'asi.6', 'asi.7', 'asi.8', 'asi.9', 'asi.10',
       'asi.11', 'asi.12', 'asi.13', 'asi.14', 'asi.15', 'asi.16', 'asi.17',
       'asi.18', 'pc.1', 'pc.2', 'pc.3', 'psp.1', 'psp.2', 'psp.3',
       'frost.ps.1', 'frost.ps.2', 'frost.ps.3', 'frost.ps.4', 'frost.com.1',
       'frost.com.2', 'frost.com.3', 'frost.com.4', 'frost.com.5',
       'frost.da.1', 'frost.da.2', 'frost.da.3', 'frost.da.4', 'stat.anx.1',
       'stat.anx.2', 'stat.anx.3', 'stat.anx.4', 'stat.anx.5', 'stat.anx.6',
       'stat.anx.7', 'stat.anx.8', 'stat.anx.9', 'stat.anx.10', 'stat.anx.11',
       'stat.anx.12', 'stat.anx.13', 'stat.anx.14', 'stat.anx.15',
       'stat.anx.16', 'math.anx.1', 'math.anx.2', 'math.anx.3', 'math.anx.4',
       'math.anx.5', 'math.anx.6', 'math.anx.7', 'math.anx.8', 'math.anx.9',
       'faculty', 'stats', 'stat

In [132]:
x = numeric.drop(columns=['math.anx', 'math.anx.1', 'math.anx.2', 'math.anx.3', 'math.anx.4', 'math.anx.5', 'math.anx.6', 'math.anx.7', 'math.anx.8', 'math.anx.9'])
y = numeric['math.anx']

In [ ]:
# numeric was not used in final result due to a weaker R2

In [142]:
train_X, val_X, train_y, val_y = train_test_split(X_encoded, y, random_state=1)

In [143]:
nums = [100, 500, 1000, 1500, 2000]

In [144]:
def get_estimators(n_estimators, train_X, train_y):
    model = RandomForestRegressor(random_state=1, n_estimators=n_estimators, max_leaf_nodes=500)
    model.fit(train_X, train_y)
    y_pred = model.predict(val_X)
    r2 = r2_score(val_y, y_pred)
    rmse = root_mean_squared_error(val_y, y_pred)
    print(r2, rmse)

In [145]:
for i in nums:
    get_estimators(i, train_X, train_y)

0.5024825372509432 0.6042486370821262
0.5060608046854023 0.6020717637779381
0.5040510672726553 0.6032953737294429
0.503550460455216 0.6035997776538911
0.5061569370237431 0.6020131721704477


In [65]:
# n_estimators = 1500

In [109]:
num = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [112]:
def get_leaf(max_depth, train_X, train_y):
    model = RandomForestRegressor(random_state=1, n_estimators=1500, max_depth=max_depth)
    model.fit(train_X, train_y)
    y_pred = model.predict(val_X)
    r2 = r2_score(val_y, y_pred)
    rmse = root_mean_squared_error(val_y, y_pred)
    print(r2, rmse)

In [113]:
for i in num:
    get_leaf(i, train_X, train_y)

0.3047037651954948 0.7141635325544474
0.39374800232631446 0.6668671469252697
0.41082350331779616 0.6574086862320847
0.4106778569032945 0.6574899380317577
0.4057730710232915 0.6602203336851387
0.40094661433842815 0.6628961469129351
0.3957535314916991 0.6657632087734519
0.3947958657902012 0.6662905814600845
0.3988632986468985 0.6640478166748357
0.39493170486744444 0.666215802247667
0.39425155912055165 0.6665901373290147
0.39455868377658354 0.6664211296937326
0.39308365175289817 0.6672324341878061
0.3931409730244946 0.6672009244790973
0.3920772521280482 0.6677854134654133
0.393227406402225 0.6671534089272954


In [ ]:
# max_depth = 3

In [ ]:
# Hyperparameter tuning

In [146]:
param = {
    'n_estimators':randint(1400,1600),
    'max_depth': randint(2,4),
    'min_samples_split': randint(6,12),
    'min_samples_leaf': randint(1,10),
    'max_features': ['sqrt', 'log2'],
}

random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=1),
    param_distributions=param,
    n_iter=50,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=1
)
random_search.fit(train_X, train_y)

,estimator,RandomForestR...andom_state=1)
,param_distributions,"{'max_depth': <scipy.stats....t 0x1304c9790>, 'max_features': ['sqrt', 'log2'], 'min_samples_leaf': <scipy.stats....t 0x13043b500>, 'min_samples_split': <scipy.stats....t 0x1304caae0>, ...}"
,n_iter,50
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,1
,error_score,nan


In [147]:
print(random_search.best_params_, random_search.best_score_)

{'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 1551} 0.5473128228706807


In [148]:
model = RandomForestRegressor(random_state=1, n_estimators=1551, max_leaf_nodes=500, oob_score=True, bootstrap=True, min_samples_leaf=1, min_samples_split=6, max_depth=3, max_features='sqrt')

In [149]:
model.fit(train_X, train_y)

,n_estimators,1551
,criterion,'squared_error'
,max_depth,3
,min_samples_split,6
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,500
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,True


In [150]:
y_pred = model.predict(val_X)

In [ ]:
# Test model on validation set

In [151]:
r2 = r2_score(val_y, y_pred)
rmse = root_mean_squared_error(val_y, y_pred)
print(r2, rmse)

0.4258612283958262 0.6491124707808684


In [152]:
model.oob_score_

0.5581370096891787